# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.

Analysis will be performed on the following dataset to determine what factors correspond to defaulting on loans across our customers. While there may be numerous componenets that contribute to if a customer defaults, I hypothesize that employment status and income affect whether or not a default occurs.

## Open the data file and have a look at the general information. 

In [1]:
# Loading all the libraries
import pandas as pd

# Load the data
df = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Task 1. Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

In [2]:
# Let's see how many rows and columns our dataset has

df.shape

(21525, 12)

In [3]:
# let's print the first 10 rows

df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In this dataset, there are a few issues that may require further investigation: `days_employed` contains negative values, `education` has casing inconsistences, and `purpose` will need to be more consistent. For example, "to have a wedding" and "having a wedding" are two distinct columns that could be condensed into "wedding". 

In [4]:
# Get info on data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


There are 21525 total rows, but `days_employed` and `total_income` only have 19351 each. The two of these columns therefore have missing values.

In [5]:
# Let's look at the filtered table with missing values in the the first column with missing data

# First, filtering to the rows with missing values in the days_employed column
null_days_employed = df[df['days_employed'].isnull()]
null_days_employed.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding


In [6]:
# Then, filtering to the rows with missing values in the total_income column
null_total_income = df[df['total_income'].isnull()]
null_total_income.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding


In [7]:
# Checking to see if these dataframes are equal
print(null_days_employed.equals(null_total_income))

True


The missing values appear to be symmetric, meaning that all the rows with missing values in `days_employed` also have missing values in `total_income` and vice versa. Therefore, if a row has a missing value in one of these columns, it will have a missing value in the other. Likewise, if a row has a non-null value in either of these columns, it will have a non-null value in the other.

In [8]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.

df[df['days_employed'].isnull() & df['total_income'].isnull()].shape

(2174, 12)

In [9]:
df_rows = 21525
df_nonnull = 19351
df_filtered_rows = 2174

print(f'Rows in filtered table matches number of missing values: {df_filtered_rows == df_rows - df_nonnull}')
print(f'Percentage of missing values: {df_filtered_rows/df_rows:.2%}')

Rows in filtered table matches number of missing values: True
Percentage of missing values: 10.10%



**Intermediate conclusion**

The number of rows in the filtered table matches the total number of missing values. There were 21525 total rows in the dataset and 19351 non-null values in our `total_income` and `days_employed` columns. We can conclude that a row will either have non-null values in both of these columns or a null value in both - there will be no cases where a row has a null in one of these two columns yet a non-null in the other.

10.10% of our data is missing. This is a large portion of the data, so instead of dropping those rows we will fill them. Multiple factors likely affect `total_income` and `days_employed`, but I hypothesize that `income_type` and `education` will have a large affect on these columns. 

Going forward, I will take a closer look at the clients with missing values in these rows to see if there's a dependency that is missing somewhere, as well as to figure out a good data point to fill the missing data with. I do not want to skew the results of the data, so I will likely choose the median or mean of another column that affects the values in `total_income` and `days_employed`.

In [10]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values

print(null_days_employed['income_type'].value_counts(normalize=True))
print(null_days_employed['education_id'].value_counts(normalize=True))
print(null_days_employed['family_status'].value_counts(normalize=True))

employee         0.508280
business         0.233671
retiree          0.189972
civil servant    0.067617
entrepreneur     0.000460
Name: income_type, dtype: float64
1    0.708372
0    0.250230
2    0.031739
3    0.009660
Name: education_id, dtype: float64
married              0.568997
civil partnership    0.203312
unmarried            0.132475
divorced             0.051518
widow / widower      0.043698
Name: family_status, dtype: float64


So far, it's hard to tell what is causing the missing data. Looking at family status, education, and income_type gives us expected distributions in the data.

**Possible reasons for missing values in data**

While it may be the case that the values are missing at random, it's too early to make that conclusion. I will need to do additional analysis to figure out if there is a pattern and compare the distribution of the filtered dataset with the distributoon of the whole dataset.

In [11]:
# Checking the distribution in the whole dataset

print(df['income_type'].value_counts(normalize=True))
print(df['education_id'].value_counts(normalize=True))
print(df['family_status'].value_counts(normalize=True))

employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
unemployed                     0.000093
entrepreneur                   0.000093
paternity / maternity leave    0.000046
student                        0.000046
Name: income_type, dtype: float64
1    0.707689
0    0.244367
2    0.034564
3    0.013101
4    0.000279
Name: education_id, dtype: float64
married              0.575145
civil partnership    0.194053
unmarried            0.130685
divorced             0.055517
widow / widower      0.044599
Name: family_status, dtype: float64


**Intermediate conclusion**

For these three columns, the distributions are extremely similar in the filtered dataset and the original dataset. This doesn't confirm that the data is missing randomly, but it does require further investigation in our data.

To investigate further, I will also be comparing the distribution of children and the distribution of gender.

In [12]:
# Check for other reasons and patterns that could lead to missing values

# Looking at the distribution of children
print(df['children'].value_counts(normalize=True))
print(null_days_employed['children'].value_counts(normalize=True))

 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64
 0     0.661914
 1     0.218491
 2     0.093836
 3     0.016559
 20    0.004140
 4     0.003220
-1     0.001380
 5     0.000460
Name: children, dtype: float64


In [13]:
# Looking at the distribution of gender
print(df['gender'].value_counts(normalize=True))
print(null_days_employed['gender'].value_counts(normalize=True))

F      0.661370
M      0.338583
XNA    0.000046
Name: gender, dtype: float64
F    0.682613
M    0.317387
Name: gender, dtype: float64


**Intermediate conclusion**

Looking at all the columns, it appears that the missing values are accidental. The distribution in the original dataset between individual columns is the same as in the filtered dataset.

**Conclusions**

The primary pattern in the dataset is how a null value in `total_income` or `days_employed` will indicate a null value in the other column as well.

10% of the data contains these missing values, so the data cannot be dropped. Instead, the data will need to be filled with a supplemental value - additional analysis will be required to find out what factors affect the non-null total_income and non_null days_employed. From that analysis, I will determine a mean or median that will be used to fill the rows.

Before filling in the missing data, the data will require extensive cleaning to take care of duplicate rows, fixing inconsistencies, and addressing unrealistic data.

## Data transformation

First, the `education` column had numerous inconsistencies, so I will standardize the spelling.

In [14]:
# Let's see all values in education column to check if and what spellings will need to be fixed

df['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [15]:
# Fixing values

df['education'] = df['education'].str.lower()

In [16]:
# Checking all the values in the column to make sure we fixed them

df['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

Next, I address the `children` column.

In [17]:
# Let's see the distribution of values in the `children` column

df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [18]:
# Looking at these values normalized
df['children'].value_counts(normalize=True)

 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64

In the children column, there are two odd values: -1 and 20. 20 (0.3% of the data) is an extremely high amount of children and may not be the actual value - it may have originated from someone counting more than just their direct children. -1 (0.2% of the data) is also an impossible value - it may have resulted from someone who did not have any kids, but we cannot confirm. Together, these make up under 1% of the data, so I will be removing these rows from the dataset.

In [19]:
# Filtering out children with age 20 and children with age -1

df = df[df['children'] != 20].reset_index(drop=True)
df = df[df['children'] != -1].reset_index(drop=True)

In [20]:
# Checking the `children` column again to make sure it's all fixed

df['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Next, checking the `days_employed` column. From the sample data we looked at initially, I know this column contains negative values, which does not make sense in the content of the column.

In [21]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentage
df['days_employed'].describe()

count     19240.000000
mean      63159.820777
std      140928.943329
min      -18388.949901
25%       -2747.235601
50%       -1203.934202
75%        -289.740178
max      401755.400475
Name: days_employed, dtype: float64

In [22]:
print("Negative columns / Total columns:", df[df['days_employed'] < 0]['days_employed'].count() / df['days_employed'].count())

Negative columns / Total columns: 0.8216735966735966


82% of the data is negative, so there may have been a technical issue while the data was being collected. The most obvious solution would be to convert these values to positive ones.

In [23]:
# Address the problematic values, if they exist

df['days_employed'] = df['days_employed'].abs()

In [24]:
# Check the result - make sure it's fixed

print("Number of negative days_employed values:", df[df['days_employed'] < 0]['days_employed'].count())
df['days_employed'].describe()

Number of negative days_employed values: 0


count     19240.000000
mean      67027.691459
std      139130.846446
min          24.141633
25%         927.984311
50%        2195.251592
75%        5556.372075
max      401755.400475
Name: days_employed, dtype: float64

Next, looking at client ages.

In [25]:
# Check the `dob_years` for suspicious values

df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

0 is showing up as an age, which is an impossible value. Perhaps this showed up as the default value for when clients did not enter their age.

In [26]:
# Check percentage of clients with age '0'
df[df['dob_years'] == 0]['dob_years'].count() / df['dob_years'].count()

0.004672460517708626

In [27]:
df['dob_years'].describe()

count    21402.000000
mean        43.300206
std         12.579055
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

About 0.5% of data has this 0 value in `dob_years`. This isn't a significant amount, but because we dropped values earlier I will fill those values with the mean of the dataset (excluding the 0). There do not appear to be outliers other than the 0 (as the mean and 50% percentile are fairly similar), so the mean instead of median can be used. The mean will be casted to an integer to keep all data types in the dob_years column consistent.

In [28]:
# Address the issues in the `dob_years` column
new_value = int(df[df['dob_years'] != 0]['dob_years'].mean())

df['dob_years'] = df['dob_years'].replace(0, new_value)

In [29]:
# Check the result - make sure it's fixed
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

Now let's check the `family_status` column.

In [30]:
# Let's see the values for the column

df['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [31]:
df['family_status'].value_counts()

married              12302
civil partnership     4160
unmarried             2799
divorced              1189
widow / widower        952
Name: family_status, dtype: int64

There are no problematic values in family_status.

Next, checking the `gender` column.

In [32]:
# Let's see the values in the column
df['gender'].value_counts()

F      14154
M       7247
XNA        1
Name: gender, dtype: int64

In [33]:
# Remove the single XNA row
df = df[df['gender'] != 'XNA']

In [34]:
# Check the result - make sure it's fixed

df['gender'].value_counts()

F    14154
M     7247
Name: gender, dtype: int64

In [35]:
df['gender'].head()

0    F
1    F
2    M
3    M
4    F
Name: gender, dtype: object

There was one row that had a XNA gender. This row was removed as no analysis could be drawn on that value.

Next, checking the `income_type` column.

In [36]:
# Let's see the values in the column
df['income_type'].value_counts()

employee                       11050
business                        5053
retiree                         3839
civil servant                   1453
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

In [37]:
# Address the problematic values, if they exist
df = df[df['income_type'].isin(['employee', 'business', 'retiree', 'civil servant'])]

In [38]:
# Check the result - make sure it's fixed

df['income_type'].value_counts()

employee         11050
business          5053
retiree           3839
civil servant     1453
Name: income_type, dtype: int64

In income_type, there were multiple values (entrepreneur, unemployed, student, and paternity / maternity leave) that each only had 1-2 rows. Because there were so few, these rows were dropped as no significant analysis could be done on them.

Next, checking for duplicates.

In [39]:
# Checking duplicates

df.duplicated().sum()

71

In [40]:
# Address the duplicates
df = df.drop_duplicates()

In [41]:
# Last check whether we have any duplicates

df.duplicated().sum()

0

In [42]:
# Check the size of the dataset that you now have after your first manipulations with it
df.shape

(21324, 12)

In [43]:
# Checking new info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21324 entries, 0 to 21401
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21324 non-null  int64  
 1   days_employed     19234 non-null  float64
 2   dob_years         21324 non-null  int64  
 3   education         21324 non-null  object 
 4   education_id      21324 non-null  int64  
 5   family_status     21324 non-null  object 
 6   family_status_id  21324 non-null  int64  
 7   gender            21324 non-null  object 
 8   income_type       21324 non-null  object 
 9   debt              21324 non-null  int64  
 10  total_income      19234 non-null  float64
 11  purpose           21324 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Originally, we had 21525 rows. The new dataset has 21324 rows, which is about a 1% change.

We corrected the values in `children` by removing impossible values and filled ages of 0 in `dob_years` with the median of the dataset. We standardized the values in `education` to have the same casing, and we removed values with 1-2 data points in `gender` and `income_type`. 

We still have null values in `days_employed` and `total_income`, which will be addressed next.

# Working with missing values

In the dataset, there are ID values for `education` (`education_id`) and `family_status` (`family_status_id`). I will create dictionaries for both of these, as these numeric values may be easier to work with within the analysis.

In [44]:
# Create the dictionaries
education_dict = dict(zip(df['education'], df['education_id']))
family_status_dict = dict(zip(df['family_status'], df['family_status_id']))

### Restoring missing values in `total_income`

`total_income` and `days_employed` have missing values. Because 10% of the data has missing values, I will be filling in these null values instead of dropping them. To do so, I will take a closer look at what factors affect these columns and use a mean or median of the associated factors to fill them.

First, I will address `total_income`. I will create a new column called age that assigns a category based on the numeric value in `dob_years`. Age is a factor that affects `total_income`, as clients with more years of experience typically make more money. This will be used to help fill the missing values in `total_income`.

In [45]:
# Function to assign an age category based on dob_years

def get_age_category(dob_years):
    if dob_years <= 29:
        return "early adulthood"
    elif dob_years <= 39:
        return "adulthood"
    elif dob_years <= 59:
        return "middle age"
    else:
        return "senior"

In [46]:
# Testing the function

print(get_age_category(29),'- early adulthood')
print(get_age_category(35), '- adulthood')
print(get_age_category(30), '- adulthood')
print(get_age_category(50), '- middle age')
print(get_age_category(60), '- senior')
print(get_age_category(70), '- senior')

early adulthood - early adulthood
adulthood - adulthood
adulthood - adulthood
middle age - middle age
senior - senior
senior - senior


In [47]:
# Creating new column based on function

df['age'] = df['dob_years'].apply(get_age_category)

In [48]:
# Checking how values in the new column

df['age'].count()

21324

In [49]:
df['age'].value_counts()

middle age         10047
adulthood           5623
early adulthood     3163
senior              2491
Name: age, dtype: int64

Income can be affected by numerous factors. Gender, level of education, and income type will individually be evaluated in order to determine which is affecting total income the most. The mean and median will be tested for both to determine if there are any outstanding large outliers skewing the data.

I will create a table  of the data that excludes missiing value. This data will be used to calculate the mean/medians that are candidates to fill the missing values.

In [50]:
# Create a table without missing values and print a few of its rows to make sure it looks fine

df_no_missing = df[df['income_type'].notnull() & df['total_income'].notnull()]
df_no_missing.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,middle age
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adulthood
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adulthood
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adulthood
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,middle age


In [51]:
# Look at the mean values for income based on age

df_no_missing.groupby('age').mean()['total_income']

age
adulthood          28322.098234
early adulthood    25528.718685
middle age         27269.997139
senior             23015.440182
Name: total_income, dtype: float64

In [52]:
# Look at the median values for income based on age

df_no_missing.groupby('age').median()['total_income']

age
adulthood          24709.4735
early adulthood    22765.2935
middle age         23434.8270
senior             19732.9470
Name: total_income, dtype: float64

It's easier to look at total_income across age now that we've put age groups into categories. The means are higher than the median, meaning that this data is right skewed and has higher outliers, so I will look at the medians instead.

There is a difference in income across age categories, notably with seniors making less money than those in adulthood. The adulthood category is younger than those in middle age, but adulthood clients makes more than middle age clients.

In [53]:
# Look at the mean values for income based on gender

df_no_missing.groupby('gender').mean()['total_income']

gender
F    24653.230890
M    30920.802064
Name: total_income, dtype: float64

In [54]:
# Look at the median values for income based on gender

df_no_missing.groupby('gender').median()['total_income']

gender
F    21462.118
M    26845.080
Name: total_income, dtype: float64

Similar to `age`, I will be looking at medians due to the right skew present in this data. Males make quite a big more than females.

In [55]:
# Look at mean values for income based on income type

df_no_missing.groupby('income_type').mean()['total_income']

income_type
business         32413.685392
civil servant    27318.902910
employee         25818.401986
retiree          21951.754892
Name: total_income, dtype: float64

In [56]:
# Look at median values for income based on income type

df_no_missing.groupby('income_type').median()['total_income']

income_type
business         27602.506
civil servant    24067.224
employee         22814.014
retiree          18956.934
Name: total_income, dtype: float64

The type of work also affects income drastically, as those working for a business make a significantly higher amount of money than those who are retired. 

In [57]:
# Look at mean values for education based on income type

df_no_missing.groupby('education').mean()['total_income']

education
bachelor's degree      33161.262038
graduate degree        27960.024667
primary education      21144.882211
secondary education    24590.262821
some college           29033.960563
Name: total_income, dtype: float64

In [58]:
# Look at median values for education based on income type

df_no_missing.groupby('education').median()['total_income']

education
bachelor's degree      28086.3810
graduate degree        25161.5835
primary education      18741.9760
secondary education    21830.2500
some college           25618.4640
Name: total_income, dtype: float64

In [59]:
df_no_missing.groupby('education').median()['total_income']
df_no_missing[df_no_missing['education'] == 'bachelor\'s degree']['total_income'].median()

28086.381

Having additional education contributes to total_income - notably, having some college experience or a bachelor's degree increases total_income.

`gender`, `income_type`, and `education` all define income the most. I will use the median of these three factors to fill the missing values to counter the high income outliers in the dataset.

In [60]:
total_income_table = df_no_missing.pivot_table(index=['education', 'income_type', 'gender'], values='total_income', aggfunc='median').reset_index()
total_income_table['total_income'] = total_income_table['total_income'].round(2)
total_income_table.head()

,education,income_type,gender,total_income
0,bachelor's degree,business,F,30420.70
1,bachelor's degree,business,M,37399.84
2,bachelor's degree,civil servant,F,25254.63
3,bachelor's degree,civil servant,M,34219.85
4,bachelor's degree,employee,F,24602.81


In [61]:
#  Function that replaces empty values based on education, income_type, and gender

def replace_missing(education, income_type, gender):
    row = total_income_table.loc[(total_income_table['education'] == education) & (total_income_table['income_type'] == income_type) & (total_income_table['gender'] == gender)]
    total_income = row.iloc[0][-1]
    return total_income

In [62]:
# Check if it works

print(replace_missing("bachelor's degree", "business", "M"), "- 37399.84")
print(replace_missing("bachelor's degree", "retiree", "M"), "- 25193.17")
print(replace_missing("primary education", "civil servant", "M"), "- 30554.67")
print(replace_missing("some college", "business", "M"), "- 30606.71")
print(replace_missing("secondary education", "civil servant", "F"), "- 20103.57")

37399.84 - 37399.84
25193.17 - 25193.17
30554.67 - 30554.67
30606.71 - 30606.71
20103.57 - 20103.57


In [63]:
# Apply it to every row
df['total_income'] = df['total_income'].fillna(df.apply(lambda x: replace_missing(x.education, x.income_type, x.gender), axis=1))

In [64]:
# Checking columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21324 entries, 0 to 21401
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21324 non-null  int64  
 1   days_employed     19234 non-null  float64
 2   dob_years         21324 non-null  int64  
 3   education         21324 non-null  object 
 4   education_id      21324 non-null  int64  
 5   family_status     21324 non-null  object 
 6   family_status_id  21324 non-null  int64  
 7   gender            21324 non-null  object 
 8   income_type       21324 non-null  object 
 9   debt              21324 non-null  int64  
 10  total_income      21324 non-null  float64
 11  purpose           21324 non-null  object 
 12  age               21324 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB


`total_income` now has 21324 values, so it has been filled.

###  Restoring values in `days_employed`

Similarly to total_income, exploratory analysis will be done on education, family status, and income type to determine which affects total_income the most.

In [65]:
# Checking education mean
df_no_missing.groupby('education').mean()['days_employed']

education
bachelor's degree       42416.968576
graduate degree        121323.630206
primary education      130340.426349
secondary education     76536.965459
some college            20761.898167
Name: days_employed, dtype: float64

In [66]:
# Checking education median
df_no_missing.groupby('education').median()['days_employed']

education
bachelor's degree      1896.866606
graduate degree        5660.057032
primary education      3043.933615
secondary education    2392.887051
some college           1209.128083
Name: days_employed, dtype: float64

There are some very extreme outliers when looking at `days_employed`, so I will only look at medians going forward.

In [67]:
# Checking family_status
df_no_missing.groupby('family_status').median()['days_employed']

family_status
civil partnership      1942.742034
divorced               2400.189289
married                2306.974767
unmarried              1467.091428
widow / widower      336995.172530
Name: days_employed, dtype: float64

In [68]:
# Checking income type
df_no_missing.groupby('income_type').median()['days_employed']

income_type
business           1555.901115
civil servant      2689.137274
employee           1572.328285
retiree          365286.622650
Name: days_employed, dtype: float64

In [69]:
# Checking age 
df_no_missing.groupby('age').median()['days_employed']

age
adulthood            1601.812856
early adulthood       999.250498
middle age           2844.547086
senior             355229.618218
Name: days_employed, dtype: float64

In [70]:
# Checking education 
df_no_missing.groupby('education').median()['days_employed']

education
bachelor's degree      1896.866606
graduate degree        5660.057032
primary education      3043.933615
secondary education    2392.887051
some college           1209.128083
Name: days_employed, dtype: float64

From this analysis, I will be using the medians of `age`, `income_type`, and `education` to fill in missing data in  `days_employed`. A pattern in the data is that older categories (ie senior in `age`, retiree in `income_type`, widow / widower in `family_status` have accumulated more days of employment than their younger counterpart. `age` very clearly shows this, with younger ages having accumulated less days of employment. `income_type` shows variability, specifically between business/employee when compared with civil servant and retiree. 

The `days_employed` column seems to have many high outliers, so again the median will be used here.

In [71]:
days_employed_table = df_no_missing.pivot_table(index=['income_type', 'age'], values='days_employed', aggfunc='median').reset_index()
days_employed_table.head()

,income_type,age,days_employed
0,business,adulthood,1536.208255
1,business,early adulthood,909.631949
2,business,middle age,1930.492791
3,business,senior,2470.912766
4,civil servant,adulthood,2590.601064


In [72]:
# Function to return a median days_employed depending on education, income_type, and age
def replace_days_employed(income_type, age):
    row = days_employed_table[(days_employed_table['income_type'] == income_type) & (days_employed_table['age']  == age)]
    days_employed = row.iloc[0][-1]
    return days_employed

In [73]:
# Check that the function works

print(replace_days_employed("business", "adulthood"), 1536.208255)
print(replace_days_employed("business", "early adulthood"), 909.631949)
print(replace_days_employed("retiree", "senior"), 365658.028700)
print(replace_days_employed("employee", "middle age"), 1988.270988)

1536.2082551132255 1536.208255
909.631949142638 909.631949
365658.02869952674 365658.0287
1988.27098829558 1988.270988


In [74]:
# Apply function to the income_type

df['days_employed'] = df['days_employed'].fillna(df.apply(lambda x: replace_days_employed(x.income_type, x.age), axis=1))

In [75]:
# Checking if there are any null values

df['days_employed'].isnull().any()

False

In [76]:
# Check the entries in all columns - make sure we fixed all missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21324 entries, 0 to 21401
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21324 non-null  int64  
 1   days_employed     21324 non-null  float64
 2   dob_years         21324 non-null  int64  
 3   education         21324 non-null  object 
 4   education_id      21324 non-null  int64  
 5   family_status     21324 non-null  object 
 6   family_status_id  21324 non-null  int64  
 7   gender            21324 non-null  object 
 8   income_type       21324 non-null  object 
 9   debt              21324 non-null  int64  
 10  total_income      21324 non-null  float64
 11  purpose           21324 non-null  object 
 12  age               21324 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB


All of our rows have the same amount of non-null values now.

## Categorization of data

The questions that need to be answered all involve loan repayment (the `debt` column, which indicates if the customer has ever defaulted on a loan). We want to individually compare this `debt` column with number of kids, (`children`), marital status (`family_status`), income level (`total_income`) and loan purpose (`purpose`).

The values in `family_status` and `debt` are straight forward so we will not categorize them, but we will categorize the other columns.

In [77]:
# Looking at children column
df['children'].value_counts()

0    14086
1     4807
2     2051
3      330
4       41
5        9
Name: children, dtype: int64

One question asks if having children affects loan repayment, so I will create a new category, `has_children` that is 0 if the client does not have children and 1 if the client has any children (1-5).

In [78]:
# Function to return if a client has children
def get_has_children(children):
    return 0 if children == 0 else 1

In [79]:
# Applying function to df
df['has_children'] = df['children'].apply(get_has_children)

Next, we will look at `total_income` and create categories like we did previously for age. There are extreme outliers for this column, so I will create categories based on the quartiles.

In [80]:
# Looking at values
df['total_income'].describe()

count     21324.000000
mean      26476.671774
std       15762.926175
min        3306.762000
25%       17191.310750
50%       23176.550000
75%       31501.426750
max      362496.645000
Name: total_income, dtype: float64

In [81]:
# dictionary so quantiles don't have to keep being calculated
income_quantiles = {
    'high': df['total_income'].quantile(0.75),
    'medium-high': df['total_income'].quantile(0.5),
    'medium': df['total_income'].quantile(0.25)
}

# Function to categorize total_income
def get_income_level(total_income):
    if total_income > income_quantiles.get('high'):
        return "High"
    elif total_income > income_quantiles.get('medium-high'):
        return "Medium-High"
    elif total_income > income_quantiles.get('medium'):
        return "Medium"
    else:
        return "Low"

In [82]:
# Apply function to data
df['income_level'] = df['total_income'].apply(get_income_level)

Finally, I will create categories for purposes.

In [83]:
# Viewing values in purpose
df['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

Looking at the unique values, I can identify five main categories: Housing, Real Estate/Property, Education, Cars, and Weddings.

In [84]:
# Creating function to assign purposes a category
def get_purpose_category(purpose):
    if 'hous' in purpose:
        return "House"
    elif 'real estate' in purpose or 'property' in purpose:
        return "Property"
    elif 'edu' in purpose or 'university' in purpose:
        return "Education"
    elif 'car' in purpose:
        return "Car"
    elif 'wedding' in purpose:
        return "Wedding"
    else:
        return "Other"

In [85]:
# Applying function
df['purpose_category'] = df['purpose'].apply(get_purpose_category)

In [86]:
# Checking values - All rows should fall into one of the categories and not "Other", so making sure there are no "Other" values
df['purpose_category'].unique()

array(['House', 'Car', 'Education', 'Wedding', 'Property'], dtype=object)

In [87]:
# Seeing how many purposes there are
df['purpose_category'].value_counts()

Property     6959
Car          4278
Education    3988
House        3787
Wedding      2312
Name: purpose_category, dtype: int64

Now that the data has been categorized, we can check our hypotheses.

## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [88]:
# Creating a dictionary that tracks number of clients that have defaulted depending on if they had children

has_children_table = df.pivot_table(index='has_children', values='debt', aggfunc='sum')
has_children_dict = has_children_table['debt'].to_dict()
has_children_dict

{0: 1063, 1: 667}

In [90]:
# Creating pivot table
pivot_table_children = df.reset_index().pivot_table(
    index='has_children',
    columns= 'debt',
    values='index',  # just to count unique users, we can use any other column. It's just as I like to do it.
    aggfunc='count'
)

# Calculating the percentage of clients that did not return the loan depending on the number of children they have
pivot_table_children['fraction'] = pivot_table_children[1] / (pivot_table_children[1] + pivot_table_children[0]) * 100

pivot_table_children.sort_values(by='fraction', ascending=True)



debt,0,1,fraction
has_children,,,
0,13023,1063,7.546500
1,6571,667,9.215253


**Conclusion**

Clients with any amount of children default approximately 2% more than clients who do not have children. Therefore, having children puts a client at a greater risk of defaulting. 

**Is there a correlation between family status and paying back on time?**

In [91]:
# Dictionary that tracks family status and number of clients who have defaulted for each
family_status_table = df.pivot_table(index='family_status', values='debt', aggfunc='sum')
family_status_dict = family_status_table['debt'].to_dict()
family_status_dict

{'civil partnership': 385,
 'divorced': 84,
 'married': 925,
 'unmarried': 273,
 'widow / widower': 63}

In [92]:
# Print chance of defaulting for each family status
for family_status, defaults in family_status_dict.items():
    category_total = len(df.loc[df['family_status'] == family_status])
    print(f'{family_status.title()} risk of defaulting: {defaults / category_total:.2%}')

Civil Partnership risk of defaulting: 9.32%
Divorced risk of defaulting: 7.06%
Married risk of defaulting: 7.55%
Unmarried risk of defaulting: 9.77%
Widow / Widower risk of defaulting: 6.62%


**Conclusion**

Clients in who are in civil partnerships or are unmarried have the highest chance of defaulting. In comparision, clients who have been married previously or who have divorced have a 2% less chance of defaulting. Widows / widowers have the lowest chance at defaulting.

Therefore, certain family statuses (civil partnerships and unmarried couples) have the highest risk associated with them.

**Is there a correlation between income level and paying back on time?**

In [93]:
# Dictionary that tracks income level and number of clients who have defaulted for each
income_level_table = df.pivot_table(index='income_level', values='debt', aggfunc='sum')
income_level_dict = income_level_table['debt'].to_dict()
income_level_dict

{'High': 377, 'Low': 424, 'Medium': 465, 'Medium-High': 464}

In [94]:
# Print chance of defaulting for each income_level
for income_level, defaults in income_level_dict.items():
    category_total = len(df.loc[df['income_level'] == income_level])
    print(f'{income_level.title()} Income risk of defaulting: {income_level_dict[income_level] / category_total:.2%}')

High Income risk of defaulting: 7.07%
Low Income risk of defaulting: 7.95%
Medium Income risk of defaulting: 8.59%
Medium-High Income risk of defaulting: 8.84%


**Conclusion**

Clients in the highest income bracket have the least likely chance of defaulting, perhaps due to having the best means of paying them back. The clients with the highest risk of defaulting are those with medium income levels and medium-high income levels. Therefore, these clients are at the highest risk of not paying back on time.

**How does credit purpose affect the default rate?**

In [95]:
# Dictionary that tracks loan purpose and number of clients who have defaulted for each
purpose_table = df.pivot_table(index='purpose_category', values='debt', aggfunc='sum')
purpose_dict = purpose_table['debt'].to_dict()
purpose_dict

{'Car': 399, 'Education': 369, 'House': 256, 'Property': 523, 'Wedding': 183}

In [96]:
# Print chance of defaulting for each purpose
for purpose, defaults in purpose_dict.items():
    category_total = len(df.loc[df['purpose_category'] == purpose])
    print(f'{purpose.title()} Loan risk of defaulting: {defaults / category_total:.2%}')

Car Loan risk of defaulting: 9.33%
Education Loan risk of defaulting: 9.25%
House Loan risk of defaulting: 6.76%
Property Loan risk of defaulting: 7.52%
Wedding Loan risk of defaulting: 7.92%


**Conclusion**

Clients taking out loans for a car or for education tend to default significantly more than other clients. In contrast, the clients who are taking out a loan for a house tend to default less.

# General Conclusion 

Before analysis could begin, the data had to be cleaned and preprocessed. Faulty values in many columns, such as `children` and `days_employed`, were removed or adjusted. There were columns with values that were only in 1-2 rows, such as in `gender` and `income_type`, that were removed since we would not have been able to draw an analysis from them due to how infrequently they appeared.

Primary problematic artifacts included negative values that showed up in `days_employed` and the missing values in `total_income` and `days_employed`. The negative values in `days_employed` was likely a technical error, so each row was converted to a positive value. The missing values consisted of 10% of the data and were likely random. As such, analysis was done to determine what factors contributed most to each of them - the median of `education` was used to assign `total_income` for missing values, and the median of `age` was used to assign missing `days_employed` rows.

The data was categorized at various points to make analysis easier: An `age` column was created based on `dob_years` that assigned an age category to the numeric value in `dob_years`. Similarly, a binary `has_children` column was created from `children`, a `income_level` column from `total_income`, and a `purpose_category` column from the `purpose` column.

When answering the posed questions, I compared the default rates to the non-defaulted rates to account for values that simply had a greater count than the others. I compared these percentages - If the default rate was higher than the non-default rate, then I could conclude that a factor led to a greater percentage of defaults. If the default rate was lower than its non-default rate, then I could conclude that factor contributed to the client being less likely to default.

Having children was the most significant factor - Having any amount of children led clients to default more than the clients that did not have children. Being married would lead a client to default less often, but unmarried clients and clients in civil partnerships tended to default more. Clients with higher incomes tended to default less while clients with a medium income tended to default more. Finally, clients tended to default more on loans when they were taking the loan out for a car or for education. Clients taking loans out for a house defaulted less than expected.